In [1]:
# pip install qiskit==0.31.0

In [392]:
depths = []
for i in tqdm(range(len(test_dataset))):
    tqc = test_dataset[i].decompose()
    if tqc.depth() not in depths:
        depths.append(tqc.depth())

100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 12133/12133 [01:02<00:00, 195.32it/s]


In [393]:
depths

[2, 5, 3, 4, 6, 7, 8, 12, 11, 22, 19, 32, 27, 42, 35, 52, 43, 62, 51]

In [ ]:
from get_test_datasets import return_dataset
test_dataset = return_dataset()

Generating QFT circuits
Generating Addition circuits


 44%|████████████████████████████████████████████▌                                                        | 4762/10795 [00:22<00:31, 190.47it/s]

In [451]:
mlp_edge_feature = tf.keras.models.load_model("models/mlp_edge_features.keras")
dnn = tf.keras.models.load_model("models/dnn.keras")

In [455]:
from mlp_graph_feature import get_graph_features,clear_dataset

In [456]:

def pred_layout(l, num_qubits):
    layout=[]
    for i in range(len(l[0])):
        layout_i = []
        for slots in l[:num_qubits]:
            if np.argmax(slots[i]) != num_qubits:
                layout_i.append(np.argmax(slots[i]))
            else:
                layout_i.append(np.nan)
        layout.append(layout_i)
    return layout

def controlla_rip(l):
    rip = []
    for i in range(len(l)):
        for j in range(i+1, len(l)):
            if l[i] == l[j]:
                if (i,j) not in rip:
                    rip.append((i,j))

    return rip

def pred_layout_diff_elem(l,num_qubits):
    layout=[]
    for i in range(len(l[0])):
        check = 0
        while check != 1:
            layout_i = []
            for slots in l[:num_qubits]:
                if np.argmax(slots[i]) != num_qubits:
                    layout_i.append(np.argmax(slots[i]))
                else:
                    layout_i.append(np.nan)
            rip = controlla_rip(layout_i)
            if rip == []:
                check = 1
            else:
                for index in rip:
                    if l[index[0]][i][layout_i[index[0]]] > l[index[1]][i][layout_i[index[1]]]:
                        l[index[1]][i][layout_i[index[1]]]=0
                    else:
                        l[index[0]][i][layout_i[index[0]]] = 0

        layout.append(layout_i)
    return layout


In [457]:
def get_labels(y):
    labels = []
    for i in range(len(y)):
        labels.append(np.where(y[i]==np.max(y[i]))[0][0])
    return labels

num_qubits = 7
last_num_qubits = len(df.columns)-num_qubits


for qc in test_dataset[8989:8990]:
    data = datetime.datetime.today() - datetime.timedelta(days=random.randint(1,150))
    backends = ['ibm_lagos','ibm_perth','ibm_nairobi']
    backend = backends[np.random.randint(0,3)]
    l = add_line(qc,backend, optimization_level=3, refresh=True, show= False, datatime=data)
    
    d={}
    for i in range(len(l[0])):
        d[str(l[0][i])] = l[1][i]
    df = pd.DataFrame(d, index=[0])
    df = clear_dataset(df, 7)
#     x = get_graph_features(df)
    x = df.iloc[:, 2:last_num_qubits].values
    print(x.shape)
#     print(x.shape)
    labels = df.iloc[:, last_num_qubits:].values
    display(df)
#     pred = (mlp_edge_feature.predict(x))
#     predicted=get_labels(np.reshape(pred,(7,7)))
    predicted = dnn.predict(x)
    
    
    pred_or = np.array(pred_layout(predicted, num_qubits))
    pred_nr = np.array(pred_layout_diff_elem(predicted,num_qubits))
    print(pred_or, pred_nr)
    print(predicted, labels[0])    
#     print(predicted==labels[0])

2023-07-12 23:13:07.484273
Backend topology: 2023-07-12 23:13:07.484273
[[0, 1], [1, 0], [1, 2], [1, 3], [2, 1], [3, 1], [3, 5], [4, 5], [5, 3], [5, 4], [5, 6], [6, 5]]


ibmqfactory.load_account:WARNING:2023-11-26 23:13:14,765: Credentials are already in use. The existing account in the session will be replaced.


Depth: [364, 364, 364, 364, 364, 364]
na
{1: Qubit(QuantumRegister(7, 'q'), 0), 2: Qubit(QuantumRegister(7, 'q'), 1), 0: Qubit(QuantumRegister(7, 'q'), 2), 3: Qubit(QuantumRegister(7, 'q'), 3), 6: Qubit(QuantumRegister(7, 'q'), 4), 5: Qubit(QuantumRegister(7, 'q'), 5), 4: Qubit(QuantumRegister(7, 'q'), 6)}
Connections: ['01', '10', '12', '13', '21', '31', '35', '45', '53', '54', '56', '65']
(1, 89)


,last_update_date,backend_name,N_qubtis,N_cx,cx_01,cx_02,cx_03,cx_04,cx_05,cx_06,...,T1_6,T2_6,readout_error_6,0,1,2,3,4,5,6
0,2023-07-12 21:21:56-07:00,ibm_lagos,7,204,14,0,0,0,0,0,...,141.629633,155.987712,0.0143,2,0,1,3,6,5,4


[[0 1 2 3 4 5 6]] [[0 1 2 3 4 5 6]]
[array([[1., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32), array([[0., 1., 0., 0., 0., 0., 0., 0.]], dtype=float32), array([[0., 0., 1., 0., 0., 0., 0., 0.]], dtype=float32), array([[0., 0., 0., 1., 0., 0., 0., 0.]], dtype=float32), array([[0., 0., 0., 0., 1., 0., 0., 0.]], dtype=float32), array([[0., 0., 0., 0., 0., 1., 0., 0.]], dtype=float32), array([[0., 0., 0., 0., 0., 0., 1., 0.]], dtype=float32)] [2 0 1 3 6 5 4]


In [399]:
import tensorflow as tf
from tensorflow import keras

In [400]:
from Dataset import *

In [435]:
loaded_model = tf.keras.models.load_model("models/mlp_edge_features.keras")

In [436]:
loaded_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 378)]             0         
_________________________________________________________________
dense (Dense)                (None, 189)               71631     
_________________________________________________________________
dense_1 (Dense)              (None, 94)                17860     
_________________________________________________________________
dense_2 (Dense)              (None, 49)                4655      
Total params: 94,146
Trainable params: 94,146
Non-trainable params: 0
_________________________________________________________________
